In [ ]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Lambda,Concatenate
from tensorflow.keras import optimizers, regularizers, applications
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model

import numpy as np
import scipy.io as sio
import scipy.stats as sst
import scipy

import random
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve,roc_auc_score

import matplotlib.pyplot as plt

import keras

In [ ]:
def monitor_evaluation(X,Y,model,isPlot=True,):
    predict = (model.predict(X)>0.5).reshape([-1])
    cm = confusion_matrix((Y>0.5).reshape([-1]),predict);
    if isPlot:
        print(cm/1000);print("predict accuracy",np.mean(predict>0.5));print("true accuracy",np.mean(Y>0.5))
    return np.mean(predict>0.5)

In [ ]:
# get auc_roc score
def get_auc(X,Y,model):
    predict = (model.predict(X)).reshape([-1])
    return roc_auc_score(Y, predict)

In [ ]:
# test it on train
## load encode feature, prob and tcp
monitor_data = sio.loadmat('./D1_test_features.mat')#cifar_test; clinic10_train; clinic10_test;
monitor_test_X = monitor_data['probability']
label = monitor_data['label'].reshape([-1])
prediction = monitor_data['prediction'].reshape([-1])
monitor_test_Y = label!=prediction

# get monitor_test_tcp
monitor_test_tcp = np.zeros((monitor_test_X.shape[0]))
for i in range(monitor_test_X.shape[0]):
    monitor_test_tcp[i]=monitor_test_X[i,int(label[i])]
# monitor_test_tcp = monitor_data['true_class_prob'].reshape([-1])
print(monitor_test_X.shape, monitor_test_Y.shape,monitor_test_tcp.shape)
# monitor_evaluation(monitor_test_X,1-monitor_test_Y,monitor_model)

In [ ]:
## evaluate the model perforamce before transfer
accuracy_before_transfer = []
for em_index in range(20):
    load_model_name = './emsemble_models/pretrained_D1_ensemble_v'+str(em_index)+'_new.h5'
#     load_model_name = 'monitor_without_image_dropout20_9k_ensemble_v'+str(em_index)+'_new.h5'
    monitor_model = models.load_model(load_model_name)
    cur_accuracy = monitor_evaluation(monitor_test_X,1-monitor_test_Y,isPlot=False,model=monitor_model)
    accuracy_before_transfer.append(cur_accuracy)
    print(em_index,end=',')

In [ ]:
# transfer data

monitorX_train,monitorX_test,monitorY_train,monitorY_test = train_test_split(monitor_test_X,
                                                                             monitor_test_tcp,
                                                                             test_size=0.99,
                                                                             random_state=42)
monitorX_train.shape, monitorY_train.shape

In [ ]:
## transfer
## evaluate the model perforamce before transfer
accuracy_afterp1_transfer = []
for em_index in range(20):
    load_model_name = './emsemble_models/pretrained_D1_ensemble_v'+str(em_index)+'_new.h5'
    monitor_model = models.load_model(load_model_name)
    
#     ## get data
#     predict_TCP = monitor_model.predict(monitor_test_X[:,-10:])
#     TCP_entrop = predict_TCP*(1-predict_TCP)*4
#     transfer_list = TCP_entrop.reshape([-1]).argsort()[::-1][:900]#[np.random.choice(1000,500)]
#     monitorX_train = monitor_test_X[transfer_list]
#     monitorY_train = monitor_test_tcp[transfer_list]
    
    # transfer step 1: control layers
    for layer in monitor_model.layers:
        layer.trainable=False
#     monitor_model.layers[-1].trainable=True
    monitor_model.layers[-2].trainable=True
    # transfer step 2: compile
    monitor_model.compile(optimizer='Adam',loss='mean_squared_error',metrics=['accuracy'])
    # transfer step 3: transfer
    history = monitor_model.fit(monitorX_train,monitorY_train,epochs=250,
                                validation_split=0.1,shuffle=True,verbose=0)
    # transfer step 4: evaluate accuracy
    cur_accuracy = monitor_evaluation(monitor_test_X,1-monitor_test_Y,isPlot=False,model=monitor_model)
    accuracy_afterp1_transfer.append(cur_accuracy)
#     print(get_auc(monitor_test_X,1-monitor_test_Y,monitor_model))
    print(em_index,end=',')